## Cloning a website using wget 
You can find detailed documentation about wget via [this link](https://www.computerhope.com/unix/wget.htm). You can also add --level=1 parameter for one-level depth scraping. I am going to clone Class Central.

In [5]:
#!wget -U --mirror --level=1 "www.example.com"
!wget -U --recursive  --page-requisites --html-extension --convert-links https://www.classcentral.com/

Both --no-clobber and --convert-links were specified, only --convert-links will be used.
--2023-02-26 14:21:08--  https://www.classcentral.com/
Resolving www.classcentral.com (www.classcentral.com)... 104.22.24.156, 104.22.25.156, 172.67.30.18, ...
Connecting to www.classcentral.com (www.classcentral.com)|104.22.24.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘www.classcentral.com/index.html’

www.classcentral.co     [   <=>              ] 649,00K  1,20MB/s    in 0,5s    

2023-02-26 14:21:09 (1,20 MB/s) - ‘www.classcentral.com/index.html’ saved [664577]

Loading robots.txt; please ignore errors.
--2023-02-26 14:21:09--  https://www.classcentral.com/robots.txt
Reusing existing connection to www.classcentral.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 340 [text/plain]
Saving to: ‘www.classcentral.com/robots.txt’

www.classcentral.co 100%[===================>]     340  --.-KB/s    in 0s      

2023-02

In [6]:
from google.cloud import translate_v2 as translate
from bs4 import BeautifulSoup

In [30]:
translate_client = translate.Client.from_service_account_json('YOUR_KEY_PATH_WITH_JSON_FILE_GOES_HERE')
target_language = 'tr'


In [24]:
import os

def scan_html_files(folder_path):
    html_files = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            if file_path.endswith(".html"):
                html_files.append(file_path)
    return html_files

# replace with the path to your folder
folder_path = "./"
html_files = scan_html_files(folder_path)

# print html files path 
print(html_files)


['./www.classcentral.com/index.html']


In [28]:
for html_to_translate in html_files:
    with open(f"{html_to_translate}", 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find('body') # I just want to change text inside body tag
    for tag in body.find_all():
        if tag.name == 'style' or tag.name == 'script':
            continue
        if not tag.string:
            continue
        translation = translate_client.translate(
            tag.string, target_language=target_language)
        tag.string.replace_with(translation['translatedText'])
    with open(html_to_translate.split('.html')[0]+'_translated.html', 'w') as f:
        f.write(str(soup))
